In [14]:
import os
from os import read
import pandas as pd

In [15]:
# read data "padded_df_std.csv"
df = pd.read_csv(os.path.join(os.getcwd(), 'red_padded_df.csv'))
df['wakeDate'] = pd.to_datetime(df['wakeDate'])
df.head()

,wakeDate,pid,duration,TST,REM:nREM,sleep,activity,stress,productivity,steps,steps_std,TST_std
0,2022-04-28,edr,31140.0,7.633333,0.275766,5.0,0.0,3.0,4.0,14713.0,1.226819,0.997918
1,2022-04-29,edr,28470.0,7.475000,0.321060,5.0,1.0,5.0,3.0,4316.0,-1.145889,0.859108
2,2022-05-02,edr,26880.0,6.916667,0.328000,5.0,1.0,4.0,3.0,3034.0,-1.438455,0.369619
3,2022-05-03,edr,25770.0,6.250000,0.315789,2.0,0.0,4.0,4.0,13436.0,0.935394,-0.214845
4,2022-05-04,edr,25980.0,6.683333,0.442446,5.0,1.0,3.0,3.0,14334.0,1.140327,0.165057


In [16]:
# get date range for each participant and number of rows
df['wakeDate'].groupby(df['pid']).agg(['min', 'max', 'count'])

,min,max,count
pid,,,
edr,2022-04-28,2022-06-16,29
egl,2022-04-28,2022-06-24,46
h9u,2022-05-07,2022-07-03,58
heh,2022-05-06,2022-06-27,53
lpz,2022-04-30,2022-06-27,49
mr1,2022-05-06,2022-06-27,31
oyb,2022-05-02,2022-06-19,23
ssg,2022-05-08,2022-06-23,29
uja,2022-04-28,2022-06-16,35


In [17]:
# # keep first 25 rows for each participant
# df = df.groupby('pid').head(25)
# df['wakeDate'].groupby(df['pid']).agg(['min', 'max', 'count'])

In [18]:
# insead of dates, use day number
df['day'] = df.groupby('pid').cumcount() + 1
# remove date column
df = df.drop(columns=['wakeDate','duration','steps'])
df.head()

,pid,TST,REM:nREM,sleep,activity,stress,productivity,steps_std,TST_std,day
0,edr,7.633333,0.275766,5.0,0.0,3.0,4.0,1.226819,0.997918,1
1,edr,7.475000,0.321060,5.0,1.0,5.0,3.0,-1.145889,0.859108,2
2,edr,6.916667,0.328000,5.0,1.0,4.0,3.0,-1.438455,0.369619,3
3,edr,6.250000,0.315789,2.0,0.0,4.0,4.0,0.935394,-0.214845,4
4,edr,6.683333,0.442446,5.0,1.0,3.0,3.0,1.140327,0.165057,5


In [19]:
# define X and Y
X = "REM:nREM"
Y = "steps_std"
Z = "stress"

# convert df so that each column is a time occaison for each participant
df2 = df.pivot(index='pid', columns='day', values=[X, Y, Z])
# rename first level of columns, REM:nREM to X, steps_std to Y, productivity to Z
df2.columns = df2.columns.set_levels(['X', 'Y', 'Z'], level=0)
df2.columns = [f'{col[0]}{col[1]}' for col in df2.columns]
df2.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,Z49,Z50,Z51,Z52,Z53,Z54,Z55,Z56,Z57,Z58
pid,,,,,,,,,,,,,,,,,,,,,
edr,0.275766,0.321060,0.328000,0.315789,0.442446,0.450567,0.322835,0.168297,0.357143,0.214414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
egl,0.230769,0.170877,0.290252,0.364393,0.339461,0.295423,0.330137,0.224658,NaN,0.314010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h9u,0.383607,0.382494,0.231041,0.265306,0.267427,0.415328,0.248844,0.287634,0.251701,0.264386,...,2.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
heh,0.179217,0.268844,0.286920,0.260870,0.319315,0.205665,0.297030,0.420645,0.178628,0.128571,...,4.0,1.0,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN
lpz,0.271981,0.215417,0.445274,NaN,0.309951,0.224299,NaN,0.226846,0.237984,0.354305,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# to csv
df2.to_csv(os.path.join(os.getcwd(), 'stress_data_crosslag.csv'))